疲劳驾驶分析：这是指连续驾驶4小时，在这期间没有一次休息时间是连续有20分钟时，记为一次疲劳驾驶
此处需要注意的是，只有连续休息20分钟才能将连续驾驶时间归零，那么，如果两个驾驶员的交接班时长只有15分钟，而每个人驾驶时长为2小时，这个算法就会出现判断错误，因此要判断同一驾驶员是否有疲劳驾驶还需要更多的特征值或者新的判断方法。

In [2]:
import numpy as np                
import pandas as pd
import seaborn as sns
import folium
import webbrowser
from folium.plugins import HeatMap
import os
import time
import datetime
import matplotlib.pyplot as plt
import math
import numba as nb

def composeTime(time1):
    time2 = datetime.datetime.strptime(time1, "%Y-%m-%d %H:%M:%S")
    time3 = time.mktime(time2.timetuple())
    time4 = int(time3)
    return time4

def hypot(x,y):
    return math.sqrt(x**2+y**2)

def file_name( user_dir):
        file_list = list()
        for root, dirs, files in os.walk(user_dir):
            for file in files:
                # if os.path.splitext(file)[1] == '.txt':
                if file.split('.')[-1]=='csv':
                    file_list.append(os.path.join(root, file))
        return file_list

path="car" 
csv=file_name(path)

Count=[]
TIME=[]

for j in range(len(csv)):
    data=pd.read_csv(csv[j])

    drive_list=[]
    sleep_list=[]
    drive_count=0
    rest_count=0

    drive_time=0
    sleep_time=0
    
    for i in range(len(data["acc_state"])-1):
        t1 = composeTime(data["location_time"][i])
        t2 = composeTime(data["location_time"][i+1])
        if data["acc_state"][i] == True:
            drive_count=drive_count+(t2-t1)
            rest_count=0
            if drive_count >= 4*60*60:              #判断是否连续驾驶4小时
                drive_time=drive_count/(60*60)     #换成小时
                drive_list.append(drive_time)      #每次连续驾驶4小时后就加入到“疲劳驾驶列表”中
            
        if data["acc_state"][i] == False:
            rest_count = rest_count+(t2-t1)
            if rest_count >= 60*20:
                sleep_time=rest_count/(60*60)
                sleep_list.append(sleep_time)
            
            
        if rest_count >= 60*20:            #当休息时间达到20分钟则将连续驾驶时间归零
            drive_count = 0
    if len(drive_list) > 0:                #当“疲劳驾驶列表”数据有1个以上（即存在疲劳驾驶情况）
        count=1
        Time=0
        flag=0
#这里需要看懂上面drive_list是如何添加数据的：当连续驾驶时间有4小时就添加，这里就是说如果连续行驶了5小时，那么从“4小时”开始就会
#每判断一次就向列表中添加一个数据。   所以只有当前一个数据大于后一个数据时，前一个数据是一次连续驾驶时间，而后一个是新的连续驾驶时间的
#第一次记录。
        for i in range(len(drive_list)-1):              
            if drive_list[i] > drive_list[i+1]:    #这种方式可以用来判断是否有两次及以上的连续驾驶
                Time=Time+drive_list[i]          #添加每次连续驾驶时间（每次都是大于4小时的疲劳驾驶行为）；计算一辆车的总连续驾驶时间
#疲劳驾驶次数。之前有一个count=1是因为最后一次的疲劳驾驶时间应该是 i+1>i(最后一项为最后一次驾驶的总时长)
                count=count+1                    
                flag=1                          #存在两次以上的连续驾驶则为1；标志号
        if flag==1:                             #当存在多次连续驾驶时，由于没记录最后一次的连续驾驶时长，所以需要加上列表的最后一个数据
            Time=Time+drive_list[-1]
        else:
            Time=drive_list[-1]            #只有一次疲劳驾驶时，总的时长为列表最后一个数据
        Count.append(count)
        TIME.append(Time)
    elif len(drive_list)==0:              #不存在疲劳驾驶现象
        Count.append(0)
        TIME.append(0)
    print("第"+str(j)+"个车分析完成")
print("finish")

Data=[]
Data.append(Count)
Data.append(TIME)


columns=["Count","Time" ]
    # list转dataframe
x=np.array(Data)
print("finish the while")
y=np.transpose(x)
print("finish the transpose")
df = pd.DataFrame(y, columns=columns)

    # 保存到本地excel
df.to_excel("car/tabel/car_pilaojiashi.xlsx", index=False)
print("save the excel")


第0个车分析完成
第1个车分析完成
第2个车分析完成
第3个车分析完成
第4个车分析完成
第5个车分析完成
第6个车分析完成
第7个车分析完成
第8个车分析完成
第9个车分析完成
finish
finish the while
finish the transpose
save the excel
